# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и исследование данных

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error

In [2]:
data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [3]:
data_0.name = 'Первый регион'
data_1.name = 'Второй регион'
data_2.name = 'Третий регион'

data_list = [data_0, data_1, data_2]

In [4]:
for data in data_list:
    print(data.name)
    display(data.sample(n=2))

Первый регион


,id,f0,f1,f2,product
75044,ykpiP,0.744622,0.775133,-1.975299,24.248122
74795,8HvcH,0.079191,0.933330,0.324976,60.045728


Второй регион


,id,f0,f1,f2,product
89004,rAdkN,2.745242,-0.99140,3.009049,80.859783
43150,UVf9u,-13.453952,-5.69928,0.999757,30.132364


Третий регион


,id,f0,f1,f2,product
5834,Cwg2J,-1.390135,0.202985,2.193131,5.184790
36459,Wtlsd,-0.574247,-3.129824,-1.425648,127.157436


In [5]:
for data in data_list:
    print(data.name)
    print(f'Кол-во дубликатов: {data.duplicated().sum()}, Кол-во NaN: {data.isna().sum().sum()}')
    print('Типы данных:')
    print(data.dtypes, '\n')

Первый регион
Кол-во дубликатов: 0, Кол-во NaN: 0
Типы данных:
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object 

Второй регион
Кол-во дубликатов: 0, Кол-во NaN: 0
Типы данных:
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object 

Третий регион
Кол-во дубликатов: 0, Кол-во NaN: 0
Типы данных:
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object 



In [6]:
for data in data_list:
    display(data.describe())

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


### Предобработка данных

- В датасетах не выявлено нулевых значений и дубликатов.
- Признаки стоит привести к единому масштабу.
- Нужно удалить столбцы с ID скважин - модели эта информация не нужна.


- Проверять корреляцию и удалять столбцы с признаками не будем, так как в условиях задачи указано, что все признаки значимы.

In [7]:
# Удалим столбец 'id'
data_0.drop(columns='id', inplace=True)
data_1.drop(columns='id', inplace=True)
data_2.drop(columns='id', inplace=True)

for data in data_list:
    display(data.sample(n=1))

,f0,f1,f2,product
11485,-0.6533,0.614343,-3.367193,36.700202


,f0,f1,f2,product
91569,3.266312,-14.328337,2.002933,53.906522


,f0,f1,f2,product
1323,2.464401,0.902874,-1.22256,117.710442


In [8]:
pd.options.mode.chained_assignment = None
scaler = StandardScaler()
numeric = ['f0', 'f1', 'f2']

# Функция принимает на вход датасет и возвращает отмасштабированные выборки
def data_split_scale(data):
    features = data.drop(columns='product')
    target = data['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, target,
                                                                                  train_size=0.75,
                                                                                  random_state=12345)
    scaler.fit(features_train[numeric])
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    return features_train, features_valid, target_train, target_valid

## Обучение и проверка модели

In [9]:
model = LinearRegression()

# Функция проводит обучение и валидацию модели, возвращая предсказания, фактические значения и отклонение
def model_train_valid(data):
    features_train, features_valid, target_train, target_valid = data_split_scale(data)
    
    model.fit(features_train, target_train)
    predictions = pd.Series(model.predict(features_valid), index=target_valid.index)
    
    rmse = mean_squared_error(target_valid, predictions) ** 0.5
    
    return predictions, target_valid, rmse

In [10]:
# Датафрейм для вывода результатов модели по регионам
model_results = pd.DataFrame(index=['Среднее предсказанных запасов', 'Среднее фактических запасов', 'RMSE модели'])

# Применяем модель к данным по регионам
pred_0, target_0, rmse_0 = model_train_valid(data_0)
model_results[data_0.name] = [pred_0.mean(), target_0.mean(), rmse_0]

pred_1, target_1, rmse_1 = model_train_valid(data_1)
model_results[data_1.name] = [pred_1.mean(), target_1.mean(), rmse_1]

pred_2, target_2, rmse_2 = model_train_valid(data_2)
model_results[data_2.name] = [pred_2.mean(), target_2.mean(), rmse_2]

pred_list = [pred_0, pred_1, pred_2]
target_list = [target_0, target_1, target_2]

model_results.round(2)

,Первый регион,Второй регион,Третий регион
Среднее предсказанных запасов,92.59,68.73,94.97
Среднее фактических запасов,92.08,68.72,94.88
RMSE модели,37.58,0.89,40.03


**Вывод**

Во втором регионе предсказанные запасы ниже остальных двух, но качество предсказаний модели гораздо выше. 

Первый и Третий регионы не сильно отличаются друг от друга как по средним запасам, так и по отклонению модели.

## Подготовка к расчёту прибыли

In [12]:
REG_BUDGET = 10000000000 #10 мрлд. - Бюджет региона на разработку скважин
EXPLORE_WELL_COUNT = 500 #Количество исследуемых скважин
BEST_WELL_COUNT = 200 #Количество лучших скважин для разработки
PROD_PRICE = 450000 #450 тыс. - Цена единицы продукта, который равен тысяче баррелей

In [13]:
WELL_BUDGET = int(REG_BUDGET / BEST_WELL_COUNT)
MIN_PROD_REQ = WELL_BUDGET / PROD_PRICE

print('Бюджет на разработку одной скважины:', WELL_BUDGET)
print('Минимальный объем продукта для безубыточности скважины:', round(MIN_PROD_REQ, 2))

Бюджет на разработку одной скважины: 50000000
Минимальный объем продукта для безубыточности скважины: 111.11


In [14]:
print('Количество скважин, обеспечивающих безубыточность по регионам.')
for i in range(0, 3):
    print(f'{data_list[i].name}:', pred_list[i][pred_list[i] > MIN_PROD_REQ].sort_values(ascending=False).count())

Количество скважин, обеспечивающих безубыточность по регионам.
Первый регион: 5388
Второй регион: 4594
Третий регион: 5234


**Вывод**

Как видно из минимального необходимого объема продукта, средие показания скважин по всем трем регионам не вытягивают их на уровень безубыточности. Однако количество безубыточных скважин намного превышает заданное количество в 200 скважин на регион.

## Расчёт прибыли и рисков 

In [15]:
# Функция принимает предсказанный объем продукта и возвращает прибыль
def income(predictions, target):
    best_predicted = predictions.sort_values(ascending=False)[: BEST_WELL_COUNT]
    best_actual_sum = target.sort_values(ascending=False)[best_predicted.index].sum()
    income = best_actual_sum * PROD_PRICE - REG_BUDGET
    
    return income

In [16]:
state = np.random.RandomState(12345)
sample_count = 1000

# Функция использует технику бутстрепа для вычисления доверительного интервала и средней прибыли
def bootstrap(predictions, target):
    samples = []
    for i in range(sample_count):
        sample = predictions.sample(n=EXPLORE_WELL_COUNT, replace=True, random_state=state)
        samples.append(income(sample, target))
    samples = pd.Series(samples)
    
    mean_income = samples.mean() / 1000000 #Средняя прибыль в миллионах
    loss_count = samples[samples < 0].count()
    loss_risk = loss_count / sample_count * 100
    
    
    lower = samples.quantile(0.025) / 1000000
    upper = samples.quantile(0.975) / 1000000
    
    print(f' Средняя прибыль со скважины: {mean_income.round(2)} млн.') 
    print(f' Доверительный интервал между {lower.round(2)} млн. и {upper.round(2)} млн.')
    print(f' Риск убытков: {loss_risk.round(2)}%')
    
    return ''

In [17]:
for i in range(0, 3):
    print(data_list[i].name)
    print(bootstrap(pred_list[i], target_list[i]))

Первый регион
 Средняя прибыль со скважины: 396.16 млн.
 Доверительный интервал между -111.22 млн. и 909.77 млн.
 Риск убытков: 6.9%

Второй регион
 Средняя прибыль со скважины: 461.16 млн.
 Доверительный интервал между 78.05 млн. и 862.95 млн.
 Риск убытков: 0.7%

Третий регион
 Средняя прибыль со скважины: 392.95 млн.
 Доверительный интервал между -112.23 млн. и 934.56 млн.
 Риск убытков: 6.5%



## Вывод

В условиях задачи указан максимальный порог риска убытков в 2.5%. Только **второй регион** вписывается в это значение и у него так же самая высокая средняя прибыль со скважины. В его доверительном интервале вообще отсутствуют отрицательная прибыль. Выбор региона очевиден.